# copied from cancer Sep 30, 2017 - 12:35

In [109]:
import pandas as pd
import numpy as np
import scipy as sp
import timeit
from scipy.cluster.hierarchy import dendrogram, linkage, to_tree
from scipy.cluster.hierarchy import ClusterNode, fcluster, inconsistent, cut_tree
import matplotlib.pylab as plt

### input parameters

In [110]:
hypo_name='hiv'
input_path='/scratch2/esadrfa/moliere/'+hypo_name +'/'
linkage_file='linkage_mat_ward_inv'

In [111]:
# d=2.6 #Cancer
# d=190 #HIV
d=2000 #HIV inverse

### load the linkage

In [112]:
start = timeit.default_timer()
print('loading {}'.format(input_path + linkage_file + '.npy'))
Z = np.load(input_path + linkage_file + '.npy')
stop = timeit.default_timer()
print("Time: %.1f seconds" % (stop - start))


loading /scratch2/esadrfa/moliere/hiv/linkage_mat_ward_inv.npy
Time: 0.0 seconds


In [113]:
len(Z)

36147

### convert the linkage to tree

In [114]:
L=to_tree(Z)

In [115]:
L.dist


29713.291087056521

In [116]:
# some setting for this notebook to actually show the graphs inline, you probably won't need this
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

### Build truncated linkage 

In [117]:
def get_linkage_info(N):
    return [N.get_left().get_id(), N.get_right().get_id(), N.dist, N.count]

In [118]:
tmpZ = []
leaves=[]
def build_linkage(root, verbose=False):
    if root.dist > d:    
        if root.get_left().dist >= d:             #recursive call
            build_linkage(root.get_left())
            tmpZ.append(get_linkage_info(root))
        else:                                     #get the leaf
            leaves.append([root.get_left().get_id(), root.get_id()])

        if root.get_right().dist >= d:
            build_linkage(root.get_right())
            tmpZ.append(get_linkage_info(root))
        else:
            leaves.append([root.get_right().get_id(), root.get_id()])

In [119]:
build_linkage(L)

In [120]:
len(tmpZ)

24

In [121]:
print(leaves) #leaf id, parent id
print(len(leaves))

[[72207, 72277], [72246, 72277], [72187, 72271], [72262, 72271], [72266, 72283], [72251, 72278], [72242, 72272], [72269, 72272], [72268, 72290], [72226, 72279], [72247, 72279], [72239, 72274], [72250, 72274], [72261, 72282], [72200, 72275], [72243, 72275], [72259, 72287], [72238, 72280], [72265, 72280], [72267, 72288], [72241, 72276], [72244, 72276], [72256, 72281], [72255, 72273], [72219, 72270], [72236, 72270]]
26


### Post Process Z

In [122]:
truncatedZ=[]
node_map={}
new_idx=0
for i in leaves:
    if(i[0] not in node_map):
        node_map[i[0]]=new_idx
        new_idx += 1

In [123]:
print(node_map)
print(len(node_map))

{72207: 0, 72246: 1, 72187: 2, 72262: 3, 72266: 4, 72251: 5, 72242: 6, 72269: 7, 72268: 8, 72226: 9, 72247: 10, 72239: 11, 72250: 12, 72261: 13, 72200: 14, 72243: 15, 72259: 16, 72238: 17, 72265: 18, 72267: 19, 72241: 20, 72244: 21, 72256: 22, 72255: 23, 72219: 24, 72236: 25}
26


In [124]:
fullZdic={}
num_points = len(Z) + 1 # len(Z)= n - 1
for i, link in enumerate(Z):
    fullZdic[num_points+i]= link

In [125]:
len(fullZdic)

36147

##### initial step for adding the leaf node to node_map

In [126]:
truncatedZ=[]
node_map={}
new_idx=0
for i in leaves:
    if(i[0] not in node_map):
        node_map[i[0]]=new_idx
        new_idx += 1

In [127]:
print(node_map)
print(len(node_map))

{72207: 0, 72246: 1, 72187: 2, 72262: 3, 72266: 4, 72251: 5, 72242: 6, 72269: 7, 72268: 8, 72226: 9, 72247: 10, 72239: 11, 72250: 12, 72261: 13, 72200: 14, 72243: 15, 72259: 16, 72238: 17, 72265: 18, 72267: 19, 72241: 20, 72244: 21, 72256: 22, 72255: 23, 72219: 24, 72236: 25}
26


##### add the new cluster id to node_map and create the truncatedZ

In [128]:
numFullData = len(Z) + 1
numLeaves = len(node_map)
cluster_id = 0
for i, link in enumerate(Z):
    if(link[2] < d):
        pass
    else:
        node_map[numFullData + i] = numLeaves + cluster_id
#         print(i, cluster_id, node_map)
        truncatedZ.append([node_map.get(link[0]), node_map.get(link[1]), link[2], link[3]])
        cluster_id += 1

In [129]:
map_new_old = {y:x for x,y in node_map.items()}
# node_map

In [130]:
truncatedZ;

In [131]:
map_new_old;

In [132]:
#work with original id(full linkage)
memo={}  
for i in range(numFullData+1):
    memo[i]=[i]
for i,link in enumerate(Z):
    left = memo.get(link[0]) 
    right = memo.get(link[1])
#     print('i:{}, link:{}, left:{}, right:{}'.format(i,link,left,right))
#     print(numFullData+1+i)
#     print(memo)
    if(left is not None and right is not None):
        memo[numFullData+1+i] = left + right


In [133]:
print(len(memo[numFullData * 2 -1]))
print(len(memo[numFullData * 2 -2]))

15812
20337


### get the clouds for each non-cluster hypo from dijk file

In [134]:
clouds_file = input_path + "allClouds.txt"
print(clouds_file)

/scratch2/esadrfa/moliere/hiv/allClouds.txt


In [ ]:
%%bash
# head -n 5 /scratch2/esadrfa/moliere/cancer/allClouds.txt

In [ ]:
start = timeit.default_timer()        
cloud_line = []
with open(clouds_file) as dijkFile:
    while(True):
        line1 = dijkFile.readline()
        line2 = dijkFile.readline()
        if(not line2):
            break
        cloud_line.append(line2.rstrip().split(' ')) # I am not sure about this
    
print('number of clouds: {}'.format(len(cloud_line)))
stop = timeit.default_timer()
print("count the number of lines and documents: %.1f seconds" % (stop - start))


### Make a new dijk file with new clusters and aggregated clouds

#### check nothing larger than 287 is in the memo (it is ok)

In [ ]:
memo;

In [ ]:
map_new_old;

In [ ]:
# for i, cluster in enumerate(memo):
# #     print(cluster)
#     for cloud_id in memo.get(cluster):
#         if cloud_id > 287:
#             print(i, cluster, cloud_id)
#             break

map_new_old: the lenght is the number of new clusters, and the mapping is used to develop the new clouds

memo: contains the cluster ids

cloud_line: moliere id in each node   

In [ ]:
out_file_name = 'clustered_clouds.txt'
print(input_path + out_file_name)

In [ ]:
def write_dijk_file(lst_doc_ids, cluster_ID):
    with open(input_path + out_file_name,'a') as f:
        f.write('Start: {} End: {} Path: 0 1 2 3 Weight: 1\n'.format(cluster_ID, cluster_ID))
        for doc_id in lst_doc_ids:
            f.write(doc_id + ' ')
        f.write('\n') 

# Final dijkstra output

In [ ]:
start = timeit.default_timer()
for new_cluster_id in map_new_old:
    old_cluster_id = map_new_old.get(new_cluster_id)
    set_doc_id = set()
#     print('cluster_id type {}'.format(type(cluster_id)))
    for line_id in memo.get(old_cluster_id):
#         print (line_id, end=',')
        if line_id <= numFullData:
            for doc_id in cloud_line[line_id-1]:
    #             print(doc_id)
                set_doc_id.add(doc_id)
        else:
            print ('line_id is for clusters', line_id)
            break
    print(new_cluster_id, len(set_doc_id))
    write_dijk_file(set_doc_id, new_cluster_id)
stop = timeit.default_timer()
print("write the documents id: %.1f seconds" % (stop - start))

In [ ]:
# some setting for this notebook to actually show the graphs inline, you probably won't need this
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation
plt.figure(figsize=(10, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
ddata = dendrogram(
    truncatedZ,
    leaf_rotation=90.,  # rotates the x axis labels
#     leaf_font_size=20.,  # font size for the x axis labels
    #labels=["a", "b", "c", "d"]
    
)

# The rest is not useful anymore - Sep 29, 2017

In [89]:
cnt = 0
for i in final_leaves:
    cnt += i[1]
print('sum all leaves:',cnt)

sum all leaves: 36148


In [12]:
max_level= 10
def preorder_traverse(G, root, c_level):
    if root.is_leaf():
        return root.get_id()
    if(c_level < max_level):
        return str(root.get_id()) + ',' + str(preorder_traverse(root.get_left(), c_level+1)) + ',' + str(preorder_traverse(root.get_right(), c_level+1)) 
    

In [107]:
max_d = 4
clusters = fcluster(Z, max_d , criterion='distance')
setC = set(clusters)
print('number of clusters {}'.format(len(setC)))
# print('size of list of clusters {}'.format(len(clusters)))

number of clusters 16


In [112]:
cluster_data_count=[]
for i in range(len(setC)):
    cluster_data_count.append(0)
for i in clusters:
    cluster_data_count[i-1] += 1

res = []
for i in range(len(setC)):
    res.append([i+1, cluster_data_count[i]])
print(res)

[[1, 15], [2, 17], [3, 22], [4, 18], [5, 12], [6, 22], [7, 12], [8, 11], [9, 8], [10, 15], [11, 15], [12, 11], [13, 54], [14, 23], [15, 11], [16, 21]]


In [113]:
truncatedZ

[[11, 12, 4.0440748811252307, 65.0],
 [7, 8, 4.1583460567452617, 19.0],
 [4, 5, 4.3958717778449872, 34.0],
 [6, 17, 4.5950143262858507, 31.0],
 [14, 15, 4.7403447064252866, 32.0],
 [18, 19, 4.9328363474081787, 65.0],
 [3, 21, 5.4042873637343565, 83.0],
 [10, 16, 5.4544297106316613, 80.0],
 [2, 22, 6.1211624976523282, 105.0],
 [0, 1, 6.2136474479915726, 32.0],
 [9, 23, 6.2392993858482182, 95.0],
 [24, 26, 6.5050671137787885, 200.0],
 [25, 27, 7.1302767606380018, 232.0],
 [13, 20, 7.7383803302348309, 55.0],
 [28, 29, 10.781118163353298, 287.0]]

In [115]:
fullZdicCheck={}
num_points = len(truncatedZ) + 1 # len(Z)= n - 1
for i, link in enumerate(truncatedZ):
    fullZdicCheck[num_points+i]= link
fullZdicCheck

{16: [11, 12, 4.0440748811252307, 65.0],
 17: [7, 8, 4.1583460567452617, 19.0],
 18: [4, 5, 4.3958717778449872, 34.0],
 19: [6, 17, 4.5950143262858507, 31.0],
 20: [14, 15, 4.7403447064252866, 32.0],
 21: [18, 19, 4.9328363474081787, 65.0],
 22: [3, 21, 5.4042873637343565, 83.0],
 23: [10, 16, 5.4544297106316613, 80.0],
 24: [2, 22, 6.1211624976523282, 105.0],
 25: [0, 1, 6.2136474479915726, 32.0],
 26: [9, 23, 6.2392993858482182, 95.0],
 27: [24, 26, 6.5050671137787885, 200.0],
 28: [25, 27, 7.1302767606380018, 232.0],
 29: [13, 20, 7.7383803302348309, 55.0],
 30: [28, 29, 10.781118163353298, 287.0]}